In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
import os
from argparse import ArgumentParser
import tensorflow_datasets as tfds
from tensorflow.keras.callbacks import TensorBoard

In [2]:
class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = Dense(embed_dim)
        self.key_dense = Dense(embed_dim)
        self.value_dense = Dense(embed_dim)
        self.combine_heads = Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)

        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        output = self.combine_heads(concat_attention)
        return output


class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, mlp_dim):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.mlp = tf.keras.Sequential(
            [
                Dense(mlp_dim, activation=tfa.activations.gelu),
                Dense(embed_dim),
            ]
        )

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        out1 = attn_output + inputs

        mlp_output = self.mlp(out1)
        return mlp_output + out1


class VisionTransformer(tf.keras.Model):
    def __init__(self, image_size, patch_size, num_layers, num_classes, d_model, num_heads, mlp_dim, channels=1):
        super(VisionTransformer, self).__init__()
        num_patches = (image_size // patch_size) ** 2
        self.patch_dim = channels * patch_size ** 2

        self.patch_size = patch_size
        self.d_model = d_model
        self.num_layers = num_layers

        self.rescale = Rescaling(1.0 / 255)
        self.pos_emb = self.add_weight(
            "pos_emb", shape=(1, num_patches + 1, d_model)
        )
        self.class_emb = self.add_weight("class_emb", shape=(1, 1, d_model))
        self.patch_proj = Dense(d_model)
        self.enc_layers = [
            TransformerBlock(d_model, num_heads, mlp_dim)
            for _ in range(num_layers)
        ]
        self.mlp_head = tf.keras.Sequential(
            [
                Dense(mlp_dim, activation=tfa.activations.gelu),
                Dense(num_classes),
            ]
        )

    def extract_patches(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patches = tf.reshape(patches, [batch_size, -1, self.patch_dim])
        return patches

    def call(self, x, training):
        batch_size = tf.shape(x)[0]
        x = self.rescale(x)
        patches = self.extract_patches(x)
        x = self.patch_proj(patches)

        class_emb = tf.broadcast_to(self.class_emb, [batch_size, 1, self.d_model])
        x = tf.concat([class_emb, x], axis=1)
        x = x + self.pos_emb

        for layer in self.enc_layers:
            x = layer(x, training)

        # First (class token) is used for classification
        x = self.mlp_head(x[:, 0])
        return x

In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

logdir = "logs"
image_size = 28
patch_size = 4
num_layers = 4
d_model = 64
num_heads = 4
mlp_dim = 128
lr = 3e-4
weight_decay = 1e-4
batch_size = 4096
epochs = 300

ds = tfds.load("mnist", as_supervised=True)

ds_train = (
    ds["train"]
    .cache()
    .shuffle(5 * batch_size)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)
ds_test = (
    ds["test"]
    .cache()
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model = VisionTransformer(
        image_size=image_size,
        patch_size=patch_size,
        num_layers=num_layers,
        num_classes=10,
        d_model=d_model,
        num_heads=num_heads,
        mlp_dim=mlp_dim,
        channels=1,
    )
    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tfa.optimizers.AdamW(learning_rate=lr, weight_decay=weight_decay),
        metrics=["accuracy"],
    )

model.fit(
    ds_train,
    validation_data=ds_test,
    epochs=epochs,
    callbacks=[TensorBoard(log_dir=logdir, profile_batch=0),],
)
model.save_weights(os.path.join(logdir, "weights", "vit.hdf5"))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Epoch 1/300
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
/home/viktor/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.3.1 and trying to load a custom op (custom_ops/activations/_activation_ops.so).
TensorFlow Addons has compiled its custom ops against TensorFlow 2.2.0, and there are no compatibility guarantees between the two versions. 
This means that you might get segfaults when loading the custom op, or other kind of low-level errors.
 If you do, do not file an issue on Github. This is a known limitation.

It might help you to fallback to pure Python ops with TF_ADDONS_PY_OPS . To do that, see https://github.com/tensorflow/addons#gpucpu-custom-ops 

You can also change the TensorFlow version installed on your system. You would need a TensorFlow version equal to or above 2.2.0 and strictly below 2.3.0.
 Note that nightly versions of TensorFlow, as well as non-pip TensorFlow li

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

15/15 [==============================] - 4s 278ms/step - loss: 0.1204 - accuracy: 0.9627 - val_loss: 0.1273 - val_accuracy: 0.9591
Epoch 59/300
15/15 [==============================] - 4s 279ms/step - loss: 0.1205 - accuracy: 0.9619 - val_loss: 0.1261 - val_accuracy: 0.9595
Epoch 60/300
15/15 [==============================] - 4s 278ms/step - loss: 0.1170 - accuracy: 0.9638 - val_loss: 0.1402 - val_accuracy: 0.9543
Epoch 61/300
15/15 [==============================] - 4s 278ms/step - loss: 0.1187 - accuracy: 0.9629 - val_loss: 0.1240 - val_accuracy: 0.9596
Epoch 62/300
15/15 [==============================] - 4s 279ms/step - loss: 0.1143 - accuracy: 0.9646 - val_loss: 0.1185 - val_accuracy: 0.9619
Epoch 63/300
15/15 [==============================] - 4s 278ms/step - loss: 0.1086 - accuracy: 0.9667 - val_loss: 0.1180 - val_accuracy: 0.9617
Epoch 64/300
15/15 [==============================] - 4s 278ms/step - loss: 0.1097 - accuracy: 0.9661 - val_loss: 0.1169 - val_accuracy: 0.9606
Epoch

Epoch 115/300
15/15 [==============================] - 4s 279ms/step - loss: 0.0782 - accuracy: 0.9751 - val_loss: 0.0904 - val_accuracy: 0.9717
Epoch 116/300
15/15 [==============================] - 4s 279ms/step - loss: 0.0705 - accuracy: 0.9779 - val_loss: 0.0894 - val_accuracy: 0.9710
Epoch 117/300
15/15 [==============================] - 4s 279ms/step - loss: 0.0685 - accuracy: 0.9788 - val_loss: 0.0889 - val_accuracy: 0.9707
Epoch 118/300
15/15 [==============================] - 4s 278ms/step - loss: 0.0671 - accuracy: 0.9795 - val_loss: 0.0916 - val_accuracy: 0.9703
Epoch 119/300
15/15 [==============================] - 4s 279ms/step - loss: 0.0683 - accuracy: 0.9788 - val_loss: 0.0923 - val_accuracy: 0.9700
Epoch 120/300
15/15 [==============================] - 4s 279ms/step - loss: 0.0663 - accuracy: 0.9801 - val_loss: 0.0929 - val_accuracy: 0.9706
Epoch 121/300
15/15 [==============================] - 4s 277ms/step - loss: 0.0658 - accuracy: 0.9799 - val_loss: 0.0933 - val_ac

15/15 [==============================] - 4s 279ms/step - loss: 0.0489 - accuracy: 0.9850 - val_loss: 0.0801 - val_accuracy: 0.9740
Epoch 172/300
15/15 [==============================] - 4s 279ms/step - loss: 0.0472 - accuracy: 0.9860 - val_loss: 0.0851 - val_accuracy: 0.9737
Epoch 173/300
15/15 [==============================] - 4s 280ms/step - loss: 0.0470 - accuracy: 0.9861 - val_loss: 0.0838 - val_accuracy: 0.9728
Epoch 174/300
15/15 [==============================] - 4s 278ms/step - loss: 0.0477 - accuracy: 0.9856 - val_loss: 0.0908 - val_accuracy: 0.9705
Epoch 175/300
15/15 [==============================] - 4s 279ms/step - loss: 0.0494 - accuracy: 0.9846 - val_loss: 0.0828 - val_accuracy: 0.9732
Epoch 176/300
15/15 [==============================] - 4s 279ms/step - loss: 0.0483 - accuracy: 0.9849 - val_loss: 0.0894 - val_accuracy: 0.9705
Epoch 177/300
15/15 [==============================] - 4s 278ms/step - loss: 0.0513 - accuracy: 0.9837 - val_loss: 0.0815 - val_accuracy: 0.9734

15/15 [==============================] - 4s 278ms/step - loss: 0.0366 - accuracy: 0.9887 - val_loss: 0.0878 - val_accuracy: 0.9732
Epoch 228/300
15/15 [==============================] - 4s 277ms/step - loss: 0.0370 - accuracy: 0.9884 - val_loss: 0.0810 - val_accuracy: 0.9740
Epoch 229/300
15/15 [==============================] - 4s 277ms/step - loss: 0.0347 - accuracy: 0.9899 - val_loss: 0.0809 - val_accuracy: 0.9740
Epoch 230/300
15/15 [==============================] - 4s 278ms/step - loss: 0.0365 - accuracy: 0.9890 - val_loss: 0.0925 - val_accuracy: 0.9705
Epoch 231/300
15/15 [==============================] - 4s 278ms/step - loss: 0.0389 - accuracy: 0.9877 - val_loss: 0.0820 - val_accuracy: 0.9747
Epoch 232/300
15/15 [==============================] - 4s 276ms/step - loss: 0.0365 - accuracy: 0.9887 - val_loss: 0.0861 - val_accuracy: 0.9727
Epoch 233/300
15/15 [==============================] - 4s 278ms/step - loss: 0.0374 - accuracy: 0.9886 - val_loss: 0.0952 - val_accuracy: 0.9713

15/15 [==============================] - 4s 277ms/step - loss: 0.0295 - accuracy: 0.9910 - val_loss: 0.0871 - val_accuracy: 0.9742
Epoch 284/300
15/15 [==============================] - 4s 279ms/step - loss: 0.0300 - accuracy: 0.9904 - val_loss: 0.0875 - val_accuracy: 0.9729
Epoch 285/300
15/15 [==============================] - 4s 278ms/step - loss: 0.0292 - accuracy: 0.9911 - val_loss: 0.0899 - val_accuracy: 0.9727
Epoch 286/300
15/15 [==============================] - 4s 278ms/step - loss: 0.0283 - accuracy: 0.9915 - val_loss: 0.0894 - val_accuracy: 0.9731
Epoch 287/300
15/15 [==============================] - 4s 277ms/step - loss: 0.0276 - accuracy: 0.9917 - val_loss: 0.0821 - val_accuracy: 0.9744
Epoch 288/300
15/15 [==============================] - 4s 278ms/step - loss: 0.0260 - accuracy: 0.9927 - val_loss: 0.0863 - val_accuracy: 0.9734
Epoch 289/300
15/15 [==============================] - 4s 278ms/step - loss: 0.0262 - accuracy: 0.9919 - val_loss: 0.0899 - val_accuracy: 0.9727